# Compare Perfromance of Different Smoothing Functions

In [1]:
from os import path
import rareeventestimation as ree
import pandas as pd
import plotly.express as px
from rareeventestimation.evaluation.constants import INDICATOR_APPROX_LATEX_NAME, BM_SOLVER_SCATTER_STYLE, MY_LAYOUT, DF_COLUMNS_TO_LATEX, LATEX_TO_HTML, WRITE_SCALE
import plotly.graph_objects as go
from IPython.display import display, Markdown
# recommended: use autoreload for development: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2


## Load Data
### Option 1: Get precomputed data online

In [2]:
# data is here: https://archive.org/details/konstantinalthaus-rareeventestimation-data
# you can got to this link and inspect the files before loading
df= pd.read_json("https://ia801504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/indicator_function_performance.json")

### Option 2: Aggregate locally precomputed data

In [3]:
## uncomment to load existing data 
## or to compile data after computing it yourself:
# data_dir = "docs/benchmarking/data/cbree_sim/indicator_functions_performance"
# df = ree.load_data(data_dir, "*")
# df.drop(columns=["index", "Unnamed: 0"], inplace=True)
# df.drop_duplicates(inplace=True)
# df = df.query("observation_window==0") \
#     .reset_index()
# df = ree.add_evaluations(df, only_success=True)
# df.to_json(path.join(data_dir, "indicator_function_performance.json"))

In [4]:
for tgt in df.cvar_tgt.unique():
    # Count proportion of unsuccessful exit status
    df_success=df.query("cvar_tgt==@tgt")\
        .groupby(["tgt_fun", "Problem"])["Message"].apply(pd.value_counts)
    df_success = pd.DataFrame(df_success)
    df_success = df_success[df_success.index.get_level_values(2)!="Success"]
    df_success["Message"] = df_success["Message"]/200
    df_success.reset_index(inplace=True)
    # Compute order of tgt_funs form best to worst
    lvl_1_order = df.query("cvar_tgt==@tgt") \
        .groupby("tgt_fun") \
        .mean() \
        .loc[:,"Success Rate"] \
        .sort_values(ascending=False) \
        .index \
        .values
    lvl_1 = [idx for idx in lvl_1_order if idx in df_success["tgt_fun"].values]
    # arange
    tbl = pd.pivot_table(df_success,
                        values="Message",
                        columns=["level_2"],
                        index=["tgt_fun", "Problem"],
                        fill_value="0 \%", 
                        aggfunc= lambda x: f"{100*x.values.item():.1f}\\%")
    tbl = tbl.reindex(lvl_1, level=0)
    tbl.index.set_levels([*map(ree.squeeze_problem_names, tbl.index.levels[1])], level=1, inplace=True) # shorter problem names
    # style and save
    tbl.columns.name=None
    tbl.index = tbl.index.set_names(names={"tgt_fun": "Approximation"}, )
    tbl = tbl.rename(columns={
            "Not Converged.":"Not Converged",
            "attempt to get argmax of an empty sequence": "No finite weights $\\bm{{w}}^n$",
            "singular matrix":"Singular $c^n$"}, index=INDICATOR_APPROX_LATEX_NAME)  
    tbl.style.to_latex(f"success_rates_tgt_{tgt}.tex",
                       multirow_align="naive",
                       #column_format="ccrRP",
                       clines="skip-last;data")
    display(tbl)# no latex display: https://github.com/mathjax/mathjax-docs/wiki/LaTeX-Tabular-environment
    # write caption
    tbl_desc = f"Exit Messages of unsuccessful runs with stopping criterion $\\Delta_{{\\text{{Target}}}} = {tgt}$. Values are proportional to 200 sample runs."
    display(Markdown(tbl_desc))
    with open(f"success_rates_tgt_{tgt}_desc.tex", "w") as file:
        file.write(tbl_desc)

/var/folders/58/0tkszzwn1ws1mnqp2xhc5t440000gp/T/ipykernel_52724/2195837642.py:26: FutureWarning: inplace is deprecated and will be removed in a future version.
  tbl.index.set_levels([*map(ree.squeeze_problem_names, tbl.index.levels[1])], level=1, inplace=True) # shorter problem names


Not Converged No finite weights $\bm{{w}}^n$  \
Approximation     Problem                                                   
$I_\text{arctan}$ FRP (d=2)        100.0\%                           0 \%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=50)        100.0\%                           0 \%   
$I_\text{alg}$    CP                 2.0\%                           0 \%   
                  FRP (d=2)         99.0\%                           0 \%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=50)        100.0\%                           0 \%   
$I_\text{sig}$    CP                 4.0\%                           0 \%   
                  FRP (d=2)        100.0\%                           0 \%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=50)        100.0\%                           0 \%   
$I_\text{tanh}$   CP                28.0\%                           0 \%   
                  FRP (d=2)         91.0\%                          0.5\%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=2)           2.5\%                          7.0\%   
                  LP (d=50)         47.0\%                           0 \%   
$I_\text{erf}$    CP               100.0\%                           0 \%   
                  FRP (d=2)        100.0\%                           0 \%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=2)          40.5\%                           0 \%   
                  LP (d=50)         86.0\%                           0 \%   
$I_\text{ReLU}$   CP               100.0\%                           0 \%   
                  FRP (d=2)        100.0\%                           0 \%   
                  FRP (d=50)       100.0\%                           0 \%   
                  LP (d=2)          99.5\%                           0 \%   
                  LP (d=50)        100.0\%                           0 \%   

                             Singular $c^n$  
Approximation     Problem                    
$I_\text{arctan}$ FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=50)            0 \%  
$I_\text{alg}$    CP                   0 \%  
                  FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=50)            0 \%  
$I_\text{sig}$    CP                   0 \%  
                  FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=50)            0 \%  
$I_\text{tanh}$   CP                   0 \%  
                  FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=2)             0 \%  
                  LP (d=50)          39.0\%  
$I_\text{erf}$    CP                   0 \%  
                  FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=2)             0 \%  
                  LP (d=50)          14.0\%  
$I_\text{ReLU}$   CP                   0 \%  
                  FRP (d=2)            0 \%  
                  FRP (d=50)           0 \%  
                  LP (d=2)             0 \%  
                  LP (d=50)            0 \%

Exit Messages of unsuccessful runs with stopping criterion $\Delta_{\text{Target}} = 1$. Values are proportional to 200 sample runs.

/var/folders/58/0tkszzwn1ws1mnqp2xhc5t440000gp/T/ipykernel_52724/2195837642.py:26: FutureWarning: inplace is deprecated and will be removed in a future version.
  tbl.index.set_levels([*map(ree.squeeze_problem_names, tbl.index.levels[1])], level=1, inplace=True) # shorter problem names


Not Converged No finite weights $\bm{{w}}^n$  \
Approximation   Problem                                                  
$I_\text{erf}$  LP (d=50)          0 \%                           0 \%   
$I_\text{tanh}$ FRP (d=2)          0 \%                          0.5\%   
                LP (d=2)           0 \%                          7.0\%   
                LP (d=50)          0 \%                           0 \%   
$I_\text{ReLU}$ FRP (d=2)        81.5\%                           0 \%   

                          Singular $c^n$  
Approximation   Problem                   
$I_\text{erf}$  LP (d=50)         14.0\%  
$I_\text{tanh}$ FRP (d=2)           0 \%  
                LP (d=2)            0 \%  
                LP (d=50)         39.0\%  
$I_\text{ReLU}$ FRP (d=2)           0 \%

Exit Messages of unsuccessful runs with stopping criterion $\Delta_{\text{Target}} = 10$. Values are proportional to 200 sample runs.

In [5]:
for cvar_tgt in df.cvar_tgt.unique():
    df_agg = ree.aggregate_df(df.query(f"cvar_tgt==@cvar_tgt"))
    for op in ["==", "<"]:
        tgt_fun_list = df_agg.groupby("tgt_fun") \
            .mean() \
            .reset_index() \
            .query(f"`Success Rate` {op} 1.0")["tgt_fun"].unique()
        if len(tgt_fun_list) > 0:
            # arrange functions
            df_acc = pd.pivot_table(df_agg.query("tgt_fun in @tgt_fun_list"),
                            values="Relative Root MSE",
                            columns=["tgt_fun"],
                            index=["Problem"]) 
            # order functions
            df_acc = df_acc.reindex(df_acc.mean().sort_values().index, axis=1)
            # style and save
            df_acc = df_acc.rename(columns=INDICATOR_APPROX_LATEX_NAME)
            df_acc.columns.name="Approximation"
            tbl = df_acc.rename(index={p: ree.squeeze_problem_names(p) for p in df_acc.index})\
                .style.format(precision=2)
            tbl.to_latex(f"accuracy_tgt_{cvar_tgt}{'_success_only' if op=='==' else '' }.tex",
                        clines="all;data")
            display(tbl) # no latex display: https://github.com/mathjax/mathjax-docs/wiki/LaTeX-Tabular-environment
            # write caption
            tbl_desc = f"Relative root mean squared error of {'successful runs with indicator function approximations that always led to convergence' if op=='==' else 'successful runs with indicator function approximations that have not always converged'} using the stopping criterion $\\Delta_{{\\text{{Target}}}} = {cvar_tgt}$."
            display(Markdown(tbl_desc))
            with open(f"accuracy_tgt_{cvar_tgt}{'_success_only' if op=='==' else '' }_desc.tex", "w") as file:
                file.write(tbl_desc)

Approximation,$I_\text{arctan}$,$I_\text{sig}$,$I_\text{alg}$,$I_\text{tanh}$,$I_\text{erf}$,$I_\text{ReLU}$
Problem,,,,,,
CP,0.02,0.02,0.02,0.03,nan,nan
FRP (d=2),nan,nan,0.04,nan,nan,nan
LP (d=2),0.02,0.02,0.02,0.05,0.14,0.19


Relative root mean squared error of successful runs with indicator function approximations that have not always converged using the stopping criterion $\Delta_{\text{Target}} = 1$.

Approximation,$I_\text{arctan}$,$I_\text{alg}$,$I_\text{sig}$
Problem,,,
CP,0.12,0.14,0.11
FRP (d=2),0.16,0.14,0.26
FRP (d=50),0.38,0.64,0.73
LP (d=2),0.14,0.15,0.07
LP (d=50),0.42,0.50,0.64


Relative root mean squared error of successful runs with indicator function approximations that always led to convergence using the stopping criterion $\Delta_{\text{Target}} = 10$.

Approximation,$I_\text{tanh}$,$I_\text{erf}$,$I_\text{ReLU}$
Problem,,,
CP,0.10,0.15,0.54
FRP (d=2),0.48,0.94,0.22
FRP (d=50),0.93,0.95,0.94
LP (d=2),0.11,0.14,0.59
LP (d=50),0.69,0.80,0.80


Relative root mean squared error of successful runs with indicator function approximations that have not always converged using the stopping criterion $\Delta_{\text{Target}} = 10$.